Exercise 1

In [ ]:
## c). LOOCV

Exercise 2

In [ ]:
## a). low bias and low variance

Exercise 3

In [ ]:
## b). low bias and high variance

Exercise 4

In [ ]:
## Regularization are ways to fix or calibrate machine learning models tominimize the adjusting loss function and 
# to prevent overfitting and underfitting.

Exercise 5

In [10]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'mckayla-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to read from s3 bucket
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

##Reading the csv file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [11]:
## Removing the observations with N/As
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [13]:
## Defining input and target variables
X = heart.drop(columns = ['TenYearCHD'], axis = 1)
Y = heart['TenYearCHD']

##Defining the folds
kf = KFold(n_splits = 5, shuffle = True)

##Defining the lists to store results
f1_score1 = list()
f1_score2 = list()

for train_idx, val_idx in kf.split(X):
    
    ##Splitting the data
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    Y_train, Y_val = Y.iloc[train_idx], Y.iloc[val_idx]
    
    ##Model 1
    
    X1 = X_train[['age','currentSmoker','totChol','sysBP','diaBP', 'BMI','heartRate','glucose']]
    X_val_1 = X_val[['age','currentSmoker','totChol','sysBP','diaBP', 'BMI','heartRate','glucose']]
    
    ##Transform the input data
    scaler = MinMaxScaler()
    X1 = scaler.fit_transform(X1)
    X_val_1 = scaler.fit_transform(X_val_1)
    
    ##Building the logistic model
    md1 = LogisticRegression().fit(X1, Y_train)
    
    ##Predicting
    pred1 = md1.predict_proba(X_val_1)[:,1]    ##Extract a column
    
    ##Changing likelihood to labels
    labels1 = np.where(pred1 < 0.25, 0, 1)
    
    ##Storing the f1_score
    f1_score1.append(f1_score(Y_val, labels1))
    
    ##Model 2
    
    X2 = X_train[['age','currentSmoker','totChol','BMI','heartRate','glucose']]
    X_val_2 = X_val[['age','currentSmoker','totChol','BMI','heartRate','glucose']]
    
    ##Transform the input data
    scaler = MinMaxScaler()
    X2 = scaler.fit_transform(X2)
    X_val_2 = scaler.fit_transform(X_val_2)
    
    ##Building the logistic model
    md2 = LogisticRegression().fit(X2, Y_train)
    
    ##Predicting
    pred2 = md2.predict_proba(X_val_2)[:,1]     ##Extract a column
    
    ##Changing likelihood to labels
    labels2 = np.where(pred2 < 0.25, 0, 1)
    
    ##Storing the f1_score
    f1_score2.append(f1_score(Y_val, labels2))
    

In [16]:
f1_score1

[0.3508771929824561,
 0.3450479233226837,
 0.321608040201005,
 0.4411764705882353,
 0.3251533742331288]

In [17]:
f1_score2

[0.26455026455026454,
 0.3202846975088968,
 0.36482084690553745,
 0.40531561461794025,
 0.31386861313868614]

In [15]:
## Computing the f1_score
f1_score1_avg = np.mean(f1_score1)
f1_score2_avg = np.mean(f1_score2)

print('The average f1_score of model 1 is', f1_score1_avg)
print('The average f1_score of model 2 is', f1_score2_avg)

The average f1_score of model 1 is 0.3567726002655017
The average f1_score of model 2 is 0.333768007344265


In [ ]:
##Based on the data above, we see that Model 1 would be better to predict TenYearCHD, as the average f1_score is greater than Model 2.